In [2]:
import xarray as xr
import numpy as np
from glob import glob
import pickle

import sys
sys.path.append('/media/eric/T51/projects_daily/projects/wmms-cc')

from downscale import *

In [2]:
# Opening observed data for testing
obs = xr.open_dataset('/media/eric/T71/data/wmms-cc/meteo/nldas/lac_nldas.nc').load()

In [3]:
# Opening modeled future data for testing
mod_fut_files = glob('/media/eric/T71/data/wmms-cc/meteo/BiasCorrected_CalAdapt/RCP45/*.nc')
mod_fut = xr.open_dataset(mod_fut_files[5])

In [4]:
# Formatting the lat/lon coordinates for modeled future data, extracting lat/lon coordinates into np array
latvals = mod_fut.lat[(mod_fut.lat <= obs.lat.max()) & (mod_fut.lat >= obs.lat.min())].values
lonvals = mod_fut.lon[(mod_fut.lon <= obs.lon.max()) & (mod_fut.lon >= obs.lon.min())].values

In [5]:
# Observed and modeled future data are on different grids. This interpolates the observed data to the 
# same grid as the modeled future data.
obsinterp = obs.interp(lat=latvals,lon=lonvals,kwargs={'bounds_error':False,'fill_value':np.nan})

In [ ]:
# Identifying which grid cells are correlated with eachother
pcarr = identify_correlation_zones(obsinterp)

In [9]:
with open('/media/eric/T71/data/wmms-cc/meteo/Loca_Downscaled/ex_pcarrs.pk','wb') as outfile:
    pickle.dump(pcarr,outfile)

In [3]:
with open('/media/eric/T71/data/wmms-cc/meteo/Loca_Downscaled/ex_pcarrs.pk','rb') as infile:
    pcarr = pickle.load(infile)

In [7]:
mod_fut_vals,mod_fut_timestamps,mod_fut_seasons,mod_fut_doy,obs_vals,obs_daily_vals,obs_daily_timestamps,obs_daily_doy,obs_day = preprocess(obsinterp,mod_fut)

In [ ]:
dsfut = loca(mod_fut_vals,mod_fut_timestamps,mod_fut_seasons,mod_fut_doy,obs_vals,obs_daily_vals,obs_daily_timestamps,obs_daily_doy,obs_day,pcarr)